# Name Generator
GAN  
v_alpha3

attempt: 3.  
modified time: 20 Apr 2023, 3:30 PM

In [ ]:
import pandas as pd
import torch
import sklearn
import matplotlib.pyplot as plt
import numpy as np


## Data Preprocessing

In [ ]:
# Downloading the preprossed CSV file

import urllib.request

url = ENV("CSV_URL")
filename = 'good_data.csv'

urllib.request.urlretrieve(url, filename)


('good_data.csv', <http.client.HTTPMessage at 0x7fd2ca9b6670>)

In [ ]:
df = pd.read_csv(filename)
df.head()

,first_name,last_name,birth_date,birth_place_state,birth_place_country,father_first_name,father_last_name,mother_first_name,mother_last_name,gender,ethnicity
0,Cole,Hauser,1975-3-22,California,USA,Cass,Warner,Wings,Hauser,m,None
1,Christian,Bale,1974-1-30,Wales,UK,Jennifer,(James),NaN,NaN,m,None
2,NaN,NaN,1987-2-9,California,USA,Donna,(Davis),Michael,Jordan,m,None
3,Nick,Offerman,1970-6-26,Illinois,USA,Cathy,Roberts,Ric,Offerman,m,None
4,Alexander,Skarsgård,1976-8-25,Stockholms län,Sweden,Stellan,Skarsgård,My,Skarsgård,m,None


In [ ]:
names = df['first_name']
names.head()

0         Cole
1    Christian
2          NaN
3         Nick
4    Alexander
Name: first_name, dtype: object

In [ ]:
len(names)

48436

### Temp Dataset

Right now, only training the model using first names, only 1 feature.   
Later it will be trained on all features like gender, oarent's names, etc.

In [ ]:
input = names #[0:1000]
len(input), input

(48436,
 0             Cole
 1        Christian
 2              NaN
 3             Nick
 4        Alexander
            ...    
 48431          Aoi
 48432        Maria
 48433        Mandy
 48434         Jane
 48435    Treshelle
 Name: first_name, Length: 48436, dtype: object)

In [ ]:
input.values[0:5]

array(['Cole', 'Christian', nan, 'Nick', 'Alexander'], dtype=object)

In [ ]:
type('john')

str

In [ ]:
# removing nan(!= Null) values from the names

clean_input = []

for name in input:
  if name != 'NaN' and name != 'nan' and type(name) == str:
    clean_input.append(name)

clean_input[0:5]

['Cole', 'Christian', 'Nick', 'Alexander', 'Pedro']

In [ ]:
def string2OrdArr(name):
  """
  Given a string, it returns a array with it's characters' ordinal values
  """
  res = []
  for c in name:
    res.append(ord(c))
  
  return res


In [ ]:
string2OrdArr('David')

[68, 97, 118, 105, 100]

In [ ]:
# converting ordinal values back to character

chr(68)

'D'

In [ ]:
ord(' ')

32

In [ ]:
ord('A'), ord('a'), ord('Z'), ord('z')

(65, 97, 90, 122)

In [ ]:
# Defining valid Alphabatical ordinal values

valid_ord_vals = [32] # for ' '
valid_ord_vals += [i for i in range(65, 90+1)]
valid_ord_vals += [i for i in range(97, 122+1)]
valid_ord_vals[0:5]

[32, 65, 66, 67, 68]

In [ ]:
# for o in valid_ord_vals:
#   print(chr(o))

In [ ]:
MAX_NAME_LEN = names.str.len().max() # Maximum length in the values of the name
MAX_NAME_LEN

15.0

### Creating custom Name encoder and decoder

In [ ]:
['ds']+[32]*3

['ds', 32, 32, 32]

In [ ]:
def nameEncorder(name, max_len=MAX_NAME_LEN):
  """
  Given name string, it returns array of ordinals with length of MAX_NAME_LEN(here 15), by appending extra white space at the end
  """
  res = []
  for c in name:
    res.append(ord(c))
  
  res += [32] * (int(max_len) - len(name)) # ord(' '))
  
  return res


In [ ]:
david_encoded = nameEncorder('David')
david_encoded

[68, 97, 118, 105, 100, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32]

In [ ]:
def nameDecoder(name_ord_arr, max_len=MAX_NAME_LEN):
  """
  Given encoded names array, it returns name string in alphabets
  """
  res = ""
  for o in name_ord_arr:
    if o != 32:
      res += chr(o)
    else:
      break
  
  return res

In [ ]:
# defining ordinal values range

ORD_RANGE_MIN = 0
ORD_RANGE_MIN = 1,114,111

In [ ]:
nameDecoder(david_encoded)

'David'

In [ ]:
encoded_names = [nameEncorder(name) for name in clean_input]
encoded_names[:5]

[[67, 111, 108, 101, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32],
 [67, 104, 114, 105, 115, 116, 105, 97, 110, 32, 32, 32, 32, 32, 32],
 [78, 105, 99, 107, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32],
 [65, 108, 101, 120, 97, 110, 100, 101, 114, 32, 32, 32, 32, 32, 32],
 [80, 101, 100, 114, 111, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32]]

### Normalizing the data

In [ ]:
# normalizing input data

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaler.fit(encoded_names)

# Normalize the input data
input_norm = scaler.transform(encoded_names)


In [ ]:
input_norm[0].shape, input_norm[0]

((15,),
 array([0.08484848, 0.35909091, 0.34703196, 0.31363636, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]))

In [ ]:
# Creating tensor of encoded names

encoded_names = torch.tensor(encoded_names, dtype=torch.float32)
encoded_names

tensor([[ 67., 111., 108.,  ...,  32.,  32.,  32.],
        [ 67., 104., 114.,  ...,  32.,  32.,  32.],
        [ 78., 105.,  99.,  ...,  32.,  32.,  32.],
        ...,
        [ 77.,  97., 110.,  ...,  32.,  32.,  32.],
        [ 74.,  97., 110.,  ...,  32.,  32.,  32.],
        [ 84., 114., 101.,  ...,  32.,  32.,  32.]])

In [ ]:
encoded_names.dtype

torch.float32

In [ ]:
BATCH_SIZE = 100

In [ ]:
# Creating data loader for training

from torch.utils.data import Dataset, DataLoader

train_dl = name_dataloader = DataLoader(encoded_names, batch_size=BATCH_SIZE, shuffle=True)


In [ ]:
for xb in train_dl:
  print("X batch Shape", xb.shape)
  break

X batch Shape torch.Size([100, 15])


### Using GPU (if available)

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

def which_device(data):
  """Returns the device name in which the data is stored"""
  return "Cuda" if data.get_device() == -1 else "CPU"

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
device = get_default_device()
device # Cuda is NVedia GPU driver

device(type='cuda')

In [ ]:
# moving train data loader to GPU if available
train_dl = DeviceDataLoader(train_dl, device)

## Building Deep Neural Network architecture

### Discriminator

input - 15, 
20, 30, 40, 35, 20, 10, 5, 
output - 1 (0/1 - Real/Fake)

In [ ]:
import torch.nn as nn

In [ ]:
discriminator = nn.Sequential(
    nn.Linear(15, 20, device=device),
    nn.ReLU(),

    nn.Linear(20, 30, device=device),
    nn.ReLU(),

    nn.Linear(30, 10, device=device),
    nn.ReLU(),

    nn.Linear(10, 5, device=device),
    nn.ReLU(),

    nn.Linear(5, 1, device=device),

    nn.Sigmoid()
  )

In [ ]:
discriminator.float()

Sequential(
  (0): Linear(in_features=15, out_features=20, bias=True)
  (1): ReLU()
  (2): Linear(in_features=20, out_features=30, bias=True)
  (3): ReLU()
  (4): Linear(in_features=30, out_features=10, bias=True)
  (5): ReLU()
  (6): Linear(in_features=10, out_features=5, bias=True)
  (7): ReLU()
  (8): Linear(in_features=5, out_features=1, bias=True)
  (9): Sigmoid()
)

In [ ]:
# Moving Discriminator to GPU
discriminator = to_device(discriminator, device)


### Generator

In [ ]:
latent_size = 15

In [ ]:
generator = nn.Sequential(
    nn.Linear(15, 20, device=device),
    nn.ReLU(),

    nn.Linear(20, 30, device=device),
    nn.ReLU(),

    nn.Linear(30, 20, device=device),
    nn.ReLU(),

    nn.Linear(20, 15, device=device),
    nn.Tanh()
)

In [ ]:
# generator.float()

In [ ]:
# Moving Generator to GPU
generator = to_device(generator, device)


In [ ]:
xb = torch.randn(latent_size, device=device) # random latent tensors
fake_names = generator(xb)
print(fake_names.shape)
fake_names

torch.Size([15])


tensor([-0.1727, -0.0312, -0.0135,  0.0445,  0.2521,  0.1895, -0.1003, -0.2175,
         0.1153, -0.1985, -0.1744,  0.1369, -0.2012, -0.0904,  0.0525],
       device='cuda:0', grad_fn=<TanhBackward0>)

### New Name Generator function

In [ ]:
import math

def generate_new_name():
  xb = torch.randn(latent_size, device=device) # random latent tensors
  fake_name = generator(xb) # Generating new name
  output_denorm = scaler.inverse_transform([fake_name.cpu().detach().numpy()]) # de-normalizing

  # Convert the output data back to characters using chr
  output_chars = []

  for x in output_denorm[0]:
    # preprocessing x
    x = int(x)
    x = abs(x)
    x = math.ceil(x)

    if x in valid_ord_vals:
      output_chars.append(chr(x))
    else:
      output_chars.append(' ')

  # Convert the list of characters back to a string
  output_str = ''.join(output_chars)
  print("Generated new name: ", output_str)
  return output_str


In [ ]:
generate_new_name()

Generated new name:      SQ         


'    SQ         '

## Training the GAN Model

### Discriminator Train function

In [ ]:
# Training the Discriminator

import torch.nn.functional as F

def train_discriminator(real_name, opt_d):
    opt_d.zero_grad() # Clear discriminator gradients

    # Pass real names through discriminator
    real_name = to_device(real_name, device)
    real_pred = discriminator(real_name)
    real_target = torch.ones(1, device=device) # 1=Real, 0=Fake

    real_loss = F.binary_cross_entropy(real_pred, real_target)
    real_score = torch.mean(real_pred).item()
    
    # Generate fake name
    latent = torch.randn(latent_size, device=device)
    fake_name = generator(latent)

    # Pass fake name through discriminator
    fake_target = torch.zeros(1, device=device)
    fake_pred = discriminator(fake_name)
    fake_loss = F.binary_cross_entropy(fake_pred, fake_target)
    fake_score = torch.mean(fake_pred).item()

    # Update discriminator weights
    loss = real_loss + fake_loss
    loss.backward()
    opt_d.step()

    return loss.item(), real_score, fake_score

### Generator train function

In [ ]:
def train_generator(opt_g):
    opt_g.zero_grad() # Clear generator gradients
    
    # Generate fake name
    latent = torch.randn(latent_size, device=device)
    fake_name = generator(latent)
    
    # Try to fool the discriminator
    pred = discriminator(fake_name)
    target = torch.ones(1, device=device)
    loss = F.binary_cross_entropy(pred, target)
    
    # Update generator weights
    loss.backward()
    opt_g.step()
    
    return loss.item()

In [ ]:
input_norm[0].dtype

dtype('float64')

In [ ]:
torch.tensor(input_norm[0], dtype=torch.double)

tensor([0.0848, 0.3591, 0.3470, 0.3136, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000], dtype=torch.float64)

In [ ]:
train_ds = torch.tensor(input_norm)

In [ ]:
train_ds[0].dtype

torch.float64

### Model train function (a.k.a. Fit function)

In [ ]:
def fit(epochs, lr, start_idx=1):
    torch.cuda.empty_cache()
    
    # Losses & scores
    losses_g = []
    losses_d = []
    real_scores = []
    fake_scores = []
    
    # Create optimizers
    opt_d = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    opt_g = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    
    for epoch in range(epochs):
        for real_name in encoded_names:
            loss_d, real_score, fake_score = train_discriminator(real_name, opt_d) # Train discriminator
            loss_g = train_generator(opt_g) # Train generator
            
        # Record losses & scores
        losses_g.append(loss_g)
        losses_d.append(loss_d)
        real_scores.append(real_score)
        fake_scores.append(fake_score)
        
        # Log losses & scores (last batch)
        print("Epoch [{}/{}], loss_g: {:.4f}, loss_d: {:.4f}, real_score: {:.4f}, fake_score: {:.4f}"
          .format(epoch+1, epochs, loss_g, loss_d, real_score, fake_score))
    
        # # Save generated images
        # save_samples(epoch+start_idx, fixed_latent, show=False)
    
    return losses_g, losses_d, real_scores, fake_scores

In [ ]:
which_device(encoded_names)

'Cuda'

### Training phase

In [ ]:
# Keeping losses and accuracy data for record

history = []

In [ ]:
%%time
history += fit(3, 1e-4)

Epoch [1/3], loss_g: 13.9610, loss_d: 0.0000, real_score: 1.0000, fake_score: 0.0000
Epoch [2/3], loss_g: 22.2632, loss_d: 0.0000, real_score: 1.0000, fake_score: 0.0000
Epoch [3/3], loss_g: 27.2070, loss_d: 0.0000, real_score: 1.0000, fake_score: 0.0000
CPU times: user 12min 37s, sys: 13.2 s, total: 12min 51s
Wall time: 12min 50s


In [ ]:
# history += fit(3, 1e-5)

In [ ]:
%%time
# history += fit(100, 1e-4)

CPU times: user 18 µs, sys: 0 ns, total: 18 µs
Wall time: 20.7 µs


In [ ]:
# Some plots

In [ ]:
# Accuracy stuff

## Saving the trained model

In [ ]:
# Define the file path for the saved model
model_path = 'name_generator(v_alpha3).pth'

# Save the entire model
torch.save(generator, model_path)


In [ ]:
# Downloading the trained model

from google.colab import files

files.download(model_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Loading the model

# generator = torch.load(model_path)


## Demo - Generating new names

In [ ]:
for _ in range(5):
  generate_new_name()

Generated new name:              t  
Generated new name:              t  
Generated new name:              t  
Generated new name:              t  
Generated new name:              t  


# EOF